# Passing input arguments to your QiskitFunction

In this document, we will learn how to pass arguments to our function.

Let's create another file with our function [./source_files/function_with_arguments.py](./source_files/function_with_arguments.py). 

Instead of having the circuit defined inside the function (like we did in the first example), we will pass it as an argument. We will also save the results, so we can access them later by calling [save_result](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.save_result.html#qiskit_serverless.core.save_result).

Here is the function:

```python
from qiskit_serverless import get_arguments, save_result
from qiskit_serverless import get_arguments, save_result
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Get the input circuit from arguments and set up a simulated backend.
arguments = get_arguments()
circuit = arguments.get("circuit")
backend = FakeAlmadenV2()

# Transpile the circuit to one optimized for the backend's hardware.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_qc = pm.run(circuit)

# Run the transpiled circuit on the sampler and get the measurement counts.
sampler = Sampler(mode=backend)
job = sampler.run([isa_qc], shots=1024)
result = job.result()[0].data.meas.get_counts()
print("\nMeasurement counts:", result)

# Save the final result for the serverless job.
save_result(result)
```

As you can see, the circuit construction is not inside the function anymore. Instead, we parse the arguments by calling the [get_arguments](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.serializers.get_arguments.html#qiskit_serverless.serializers.get_arguments) function.

First, we will create circuit that we want to pass as an argument to the function.

In [1]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

Now let's create and configure our client

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [2]:
from qiskit_serverless import ServerlessClient
import os

client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    # If you are using the kubernetes approach the URL must be http://localhost
)
client

<gateway-client>

In [3]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="function-with-arguments",
    entrypoint="function_with_arguments.py",
    working_dir="./source_files/",
)

client.upload(function)

QiskitFunction(function-with-arguments)

Run the function using the `run` method and by passing arguments.

In [4]:
my_function = client.get("function-with-arguments")
my_function

QiskitFunction(function-with-arguments)

In [5]:
job = my_function.run(circuit=circuit)
job

<Job | 70ca2391-c803-4bdb-9b11-ff1573dd3551>

Retrieve the results from the client

In [6]:
job.result()

{'11': 545, '00': 479}